In [1]:
# We import all our dependencies.
from n2v.models import N2VConfig, N2V
import numpy as np
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from matplotlib import pyplot as plt
from keras import optimizers
import urllib
import os
import zipfile
import tensorflow as tf

/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: 

In [2]:
#freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
import keras.backend as K
K.set_learning_phase(1)
        

In [3]:
X = np.random.uniform(-2,2,(10, 96, 96, 1))

In [4]:
# You can increase "train_steps_per_epoch" to get even better results at the price of longer computation. 
config = N2VConfig(X, unet_kern_size=3, 
                   train_steps_per_epoch=5,train_epochs=5, train_loss='mse', batch_norm=True, 
                   train_batch_size=128, n2v_perc_pix=1.6, n2v_patch_shape=(64, 64), 
                   n2v_manipulator='uniform_withCP', n2v_neighborhood_radius=5)

# Let's look at the parameters stored in the config-object.
vars(config)

{'means': ['-0.0029069487592312607'],
 'stds': ['1.1533926996216468'],
 'n_dim': 2,
 'axes': 'YXC',
 'n_channel_in': 1,
 'n_channel_out': 1,
 'unet_residual': False,
 'unet_n_depth': 2,
 'unet_kern_size': 3,
 'unet_n_first': 32,
 'unet_last_activation': 'linear',
 'unet_input_shape': (None, None, 1),
 'train_loss': 'mse',
 'train_epochs': 5,
 'train_steps_per_epoch': 5,
 'train_learning_rate': 0.0004,
 'train_batch_size': 128,
 'train_tensorboard': True,
 'train_checkpoint': 'weights_best.h5',
 'train_reduce_lr': {'factor': 0.5, 'patience': 10},
 'batch_norm': True,
 'n2v_perc_pix': 1.6,
 'n2v_patch_shape': (64, 64),
 'n2v_manipulator': 'uniform_withCP',
 'n2v_neighborhood_radius': 5,
 'probabilistic': False}

In [5]:
# a name used to identify the model
model_name = 'n2v_2D_2'
# the base directory in which our model will live
basedir = 'models'
# We are now creating our network model.
model = N2V(config, model_name, basedir=basedir)

Instructions for updating:
Colocations handled automatically by placer.


/home/random/.local/lib/python3.6/site-packages/n2v/models/n2v_standard.py:409: UserWarning: output path for model already exists, files may be overwritten: /media/data/Development/imagej/project/CSBDeep/CSBDeep-N2V/src/main/python/models/n2v_2D_2
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


In [6]:
#optimizer = optimizers.SGD(lr=0.01)
#isinstance(optimizer, optimizers.Optimizer) or print(ValueError())

tfoptimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-7)
model.prepare_for_training(optimizer = optimizers.TFOptimizer(tfoptimizer))
model.keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, None, None, 1 0                                            
__________________________________________________________________________________________________
down_level_0_no_0 (Conv2D)      (None, None, None, 3 320         input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         down_level_0_no_0[0][0]          
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
down_level

In [7]:
loss = tf.get_default_graph().get_tensor_by_name("loss/mul:0")
train_op = tfoptimizer.minimize(loss, name='train')
tf.global_variables_initializer()
tf.train.Saver().as_saver_def()

filename_tensor_name: "save/Const:0"
save_tensor_name: "save/control_dependency:0"
restore_op_name: "save/restore_all"
max_to_keep: 5
keep_checkpoint_every_n_hours: 10000.0
version: V2

In [8]:
# save for training
#writer = tf.summary.FileWriter("logs", tf.get_default_graph())
with open('../resources/graph.pb', 'wb') as f:
    f.write(tf.get_default_graph().as_graph_def().SerializeToString())

In [15]:
# save for prediction
import shutil
#shutil.rmtree("../resources/prediction")
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    tf.tables_initializer().run()
    var_ab = tf.variables_initializer([a, b], name="a_and_b")
    sess.run(var_ab)
    #sess.run(tf.variables_initializer([tf.compat.v1.get_variable("conv2d_1/bias:0")]))
    sess.run(init)
    sess.run("init")
    model_input = tf.compat.v1.saved_model.build_tensor_info(sess.graph.get_tensor_by_name('input:0'))
    model_output = tf.compat.v1.saved_model.build_tensor_info(sess.graph.get_tensor_by_name('activation_11/Identity:0'))

    signature_definition = tf.compat.v1.saved_model.build_signature_def(
        inputs={"input": model_input},
        outputs={"output": model_output},
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME)

    builder = tf.compat.v1.saved_model.Builder("../resources/prediction")
    builder.add_meta_graph_and_variables(sess, [tf.compat.v1.saved_model.tag_constants.SERVING], signature_def_map={
        tf.compat.v1.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_definition
    }, clear_devices=True)
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../resources/prediction/saved_model.pb
